# Imports

In [ ]:
import os
import selenium.webdriver as webdriver
from selenium.webdriver.edge.service import Service 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import datetime

# Parameters

In [ ]:
betfair_url = 'https://www.betfair.com/exchange/plus/pt/futebol-apostas-1/today' #TODAY
#betfair_url = 'https://www.betfair.com/exchange/plus/pt/futebol-apostas-1/tomorrow' #TOMORROW

betfair_email = '' #FILL WITH YOUR CREDENTIALS
betfair_password = '' #FILL WITH YOUR CREDENTIALS

base_xpath = '//*[@id="main-wrapper"]/div/div[2]/div/ui-view/ui-view/div/div/div/div/div[1]/div/div[1]/bf-super-coupon/main/ng-include[3]/section'

oddsheaders = ["BH", "LH", "BD", "LD", "BA", "LA"]

day_of_games = "2023-12-17"
output_file = 'dados_betfair.xlsx'

# Functions

# Main Code

### Opening Betfair

In [ ]:
#GET EDGE WEBDRIVER
edge_driver_path = os.path.join(os.getcwd(), 'msedgedriver.exe')
edge_service = Service(edge_driver_path)

#OPEN BROWSER SESSION
browser = webdriver.Edge(service = edge_service)
browser.get(betfair_url) 
browser.maximize_window()

#WAIT FOR COOKIES POPUP
accepted = 0
while accepted == 0:
    try:
        AcceptCookiesButton = browser.find_element(By.ID, 'onetrust-accept-btn-handler')
        AcceptCookiesButton.click()
        accepted = 1
        print("Cookies accepted!")
    except:
        print("Waiting for cookies...")
        time.sleep(2) 

#LOGING IN
BetfairEmailInput = browser.find_element(By.ID, 'ssc-liu')
BetfairEmailInput.clear()
BetfairEmailInput.send_keys(betfair_email)
BetfairPasswordInput = browser.find_element(By.ID, 'ssc-lipw')
BetfairPasswordInput.clear()
BetfairPasswordInput.send_keys(betfair_password)
BetfairLoginButton = browser.find_element(By.ID, 'ssc-lis')
BetfairLoginButton.click()
print("Successfully logged in!")
time.sleep(5)

### Looping through all the games

In [ ]:
data = []
timestamp = datetime.datetime.now()
timestampstr = str(timestamp)

#SET FIRST PAGE
pages_to_loop = True
current_page = browser.current_url

#ENTER LOOP FOR EACH PAGE
while pages_to_loop:
    print(f'Buscando dados da página {current_page}')
    time.sleep(2)
    
    #ENTER LOOP FOR EACH LEAGUE IN PAGE
    leagues_left = True
    league_counter = 1
    while leagues_left:
        
        try:
            LigaElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[1]/div/span')
            print(f'Capturando {LigaElement.text}')
            
            #ENTER LOOP FOR EACH GAME IN LEAGUE
            games_left = True
            games_counter = 1
            while games_left:
                row = {}
                try:
                    
                    row["RunTimestamp"] = timestampstr
                    
                    #ATTEMPT TO GET DATE
                    try:
                        row["GameTimestamp"] = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[1]/a/event-line/section/bf-livescores/section/div/div/data-bf-livescores-start-date/ng-include/div/div').text
                    #OTHERWISE RETURN LIVE
                    except:
                        row["GameTimestamp"] = "Ao Vivo"
                    
                    #GET TEAMS AND ODDS
                    row["HomeTeam"] = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[1]/a/event-line/section/ul[1]/li[1]').text
                    row["AwayTeam"] = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[1]/a/event-line/section/ul[1]/li[2]').text
                    BackHomeElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[1]/ours-price-button[1]/button/label[1]').text
                    LayHomeElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[1]/ours-price-button[2]/button/label[1]').text
                    BackDrawElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[2]/ours-price-button[1]/button/label[1]').text
                    LayDrawElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[2]/ours-price-button[2]/button/label[1]').text
                    BackAwayElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[3]/ours-price-button[1]/button/label[1]').text
                    LayAwayElement = browser.find_element(By.XPATH, base_xpath + f'[{str(league_counter)}]/div[2]/bf-coupon-table/div/table/tbody/tr[{str(games_counter)}]/td[2]/div[3]/ours-price-button[2]/button/label[1]').text
                    
                    #TRANSFORM ODDS
                    oddstext = [BackHomeElement, LayHomeElement, BackDrawElement, LayDrawElement, BackAwayElement, LayAwayElement]
                    for j in range(0,6):
                        try:
                            row[oddsheaders[j]] = float(oddstext[j])
                        except:
                            row[oddsheaders[j]] = 0.0
    
                    #APPEND GAME AS ROW
                    data.append(row)
                    print(row)
                    games_counter += 1
                    time.sleep(2)
                    
                #EXIT GAME LOOP
                except:
                    print("Essa liga não tem mais jogos!")
                    games_left = False
                    time.sleep(2)
                
            league_counter += 1
            time.sleep(2)
            
        #EXIT LEAGUE LOOP
        except:
            print("Não há mais ligas nessa página!")
            leagues_left = False
            time.sleep(2)
    
    #MOVE TO NEXT PAGE
    previous_page = current_page
    NextButton = browser.find_element(By.XPATH, '//*[@id="main-wrapper"]/div/div[2]/div/ui-view/ui-view/div/div/div/div/div[1]/div/div[1]/bf-super-coupon/main/ng-include[4]/bf-coupon-page-navigation/ul/li[4]/a/label')
    NextButton.click()
    current_page = browser.current_url
    
    #IF LAST PAGE, END LOOP
    if previous_page == current_page:
        pages_to_loop = False
    time.sleep(2)

### Creating Betfair data dataframe

In [ ]:
df = pd.DataFrame.from_dict(data)
df['Dia'] = day_of_games
df['ID'] = df['HomeTeam'] + ' vs ' + df['AwayTeam'] + ' ' + df['Dia']
df.to_excel(output_file)
print("Excel exported!")

### Closing Edge Webdriver

In [ ]:
browser.close()
browser.quit()
print("Browser closed!")